In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)


Toggle code

## Receipt.ID
### Hierarchical text item classification
Taxonomic classification, categorize items according to a pre-defined taxonomy. The goal is to assign one or more categories in the taxonomy to an item. It is a multi-class **and** multi-label classification problem with hierarchical relationships between each node in the tree.

#### Items
- Items come from a wide range for categories like Produce, Meat, Beverage, Supplies. 
- Example item to category mapping:


|item|mapping|
|---|---|
|Kale  | "Food/Produce/Kale"  |
|Vinegar white wine 50 grain  | "Food/Dry-Grocery/Vinegars/White Wine Vinegar"  |
|Imported nat flank steak  | "Food/Meats/Beef/Flank Steak"  |


To solve this problem, I will undertake the following course of action:
1. Explore the dataset
    - Explore the dataset to ensure its integrity and understand the context. 
2. Identify features that may be used. 
    - If possible, engineer features that might provide greater discrimination.
3. Build k independent *text-based* classifiers for the text-based features and feed the output from these classifiers into the next layer classifier which takes in the other features. Explore a couple of classifiers that might be well suited for the problem at hand.
    - Decision Trees
    - SVM
    - AdaBoost
4.  Select appropriate classifier based on evaluation metric and tune it for optimality.

In this notebook I do processes 3 and 4.




In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Import libraries
from __future__ import absolute_import, division, print_function

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pickle
import json

# Graphing Libraries
import matplotlib.pyplot as pyplt
import seaborn as sns
sns.set_style("white")   

In [4]:

df = pd.read_pickle('data/df_data_vectors.dat')
df_catergory_lookup = pd.read_pickle('data/data_category_lookup.dat')


In [5]:
len(df)

127108

### Definition of helper functions

In [6]:
from sklearn.model_selection import train_test_split

def shuffle_split_data(X, y):
    """ Shuffles and splits data into 75% training and 25% testing subsets,
        then returns the training and testing subsets. """
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=num_train, random_state=42)
    
    # Return the training and testing data subsets
    return X_train, y_train, X_test, y_test

In [7]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(word2vec.itervalues().next())
    
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    

In [8]:

def get_categories(level, num_data = 1000):
    
    the_categories = {}
    counts = df.groupby(level).size()
    total = df.groupby(level).size().sum()
    
    for i in counts.index:
        if counts[i] > num_data:
            the_categories[int(i)] = counts[i]


    print(len(the_categories), 'categories with enough data\n')  

    for key in the_categories:
        find_name = df_catergory_lookup.category_id == key
        name = df_catergory_lookup[find_name].category_name
        the_index = name.index[0]

        #print ("{0}: {1}".format(key, name.ix[the_index]))
        
    return the_categories, total




## Current approach
The current approach used focused exclusively on the item's name, for example, a data point in the dataset would have an object's name as "Strauss Yogurt" or "Organic spinach." The first challenge with this approach lay in the fact that the item label was quite short, roughly about four to eight words.  Further, when modifiers like *gluten free*, *organic*, or *pesticide free* wherein the item's label, this added a layer of misinformation causing items like *organic milk* and *organic beer* to be classified in the same class.

## Samples
I have selected these samples to see that I can correctly separate them even though they have the modifiers in their item name


In [9]:
indices = [274, 64391, 55697] 

# Create a DataFrame of the chosen samples
samples = pd.DataFrame(df.loc[indices], columns = df.keys())
print ("Chosen samples of items:")
display(samples)

Chosen samples of items:


,mapped_category,item_id,price_stddev,primary_unit,price_mean,category,vendor_id,item_name,level_0,level_1,...,catg_986,catg_989,catg_998,catg_999,catg_1002,catg_495,catg_499,catg_1015,catg_1019,catg_1021
274,"[Food, Produce, Spinach]",1616,0.539695,1,12.6064,"[1, 9, 91]",14,organic baby spinach,1,9.0,...,0.011264,0.184686,0.235981,0.247730,0.230250,0.290698,0.240510,NaN,NaN,NaN
64391,"[Beverages, Alcoholic, Beers]",508958,None,4,165,"[3, 6, 16]",248,eel river organic 15.5,3,6.0,...,0.316517,NaN,NaN,NaN,NaN,0.267325,0.247167,0.01175,0.252006,0.307585
55697,"[Food, Meats, Beef, Beef Tongue]",455407,0.0151186,3,2.98429,"[1, 7, 70, 1675]",4218,mary's organic fryers,1,7.0,...,0.265593,0.219891,0.352890,0.281422,0.247474,0.361146,0.338465,NaN,NaN,NaN


## My Approach
I took an entirely different approach. I got inspiration from the approach that Google, [YouTube](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/36411.pdf) used in organizing videos and decided to shift the unit of analysis from item *name* to the *categories* themselves. My approach combines information from both the text-based labels as well as the item's metadata.

This method achieves two crucial things. First, by focusing on individual categories, each time a new category of item is added to the restaurant domain, instead of having to retrain the classifier on the entire dataset, all we have to do is gather enough data for that category, and train a classifier for it. This way, the approach can scale beautifully as the taxonomy grows. Second, moving the unit of analysis from text labels to categories, it becomes easier to correctly separate "organic cream" and "organic beer." 

I chose to discard categories that had less than 300 datapoints. As the datapoints in the category grows, those categories can then been trained individually.

In [10]:
the_level = {}
num_data = 300

level = 'level_0'
the_level[level], total = get_categories(level, num_data)

6 categories with enough data



In [11]:
level = 'level_1'
the_level[level], total = get_categories(level, num_data)

19 categories with enough data



In [12]:
level = 'level_2'
df.loc[:,('label')] = df.loc[:, level]
the_level[level], total = get_categories(level, num_data)

65 categories with enough data



In [13]:
level = 'level_3'
df.loc[:,('label')] = df.loc[:, level]
the_level[level], total = get_categories(level, num_data)

42 categories with enough data



In [14]:
level = 'level_4'
df.loc[:,('label')] = df.loc[:, level]
the_level[level], total = get_categories(level, num_data)

9 categories with enough data



In [15]:
k_categories = 0
for key in the_level.keys():
    k_categories += len(the_level[key].keys())
   
    
print('processed {} categories in all'.format(k_categories))

processed 141 categories in all


### Labeling
For each data point, I assigned the deepest class node in the tree to which it belonged as its label. For example, an item named "bacon ends" belonged to classes [Food, Meats, Pork]. For such an item, I assigned it is label as "Pork."

In [16]:
null_examples = df['label'].isnull()
df.ix[df[null_examples].index, 'label'] = df.loc[:, 'level_4']

In [17]:
null_examples = df['label'].isnull()
df.ix[df[null_examples].index, 'label'] = df.loc[:, 'level_3']

In [18]:
null_examples = df['label'].isnull()
df.ix[df[null_examples].index, 'label'] = df.loc[:, 'level_2']

In [19]:
null_examples = df['label'].isnull()
df.ix[df[null_examples].index, 'label'] = df.loc[:, 'level_1']

In [20]:
null_examples_ = df['label'].isnull()
df.ix[df[null_examples_].index, 'label'] = df.loc[:, 'level_0']

In [21]:
df[df['label'].isnull()]

,mapped_category,item_id,price_stddev,primary_unit,price_mean,category,vendor_id,item_name,level_0,level_1,...,catg_986,catg_989,catg_998,catg_999,catg_1002,catg_495,catg_499,catg_1015,catg_1019,catg_1021


### Retrieve the *k* training sets for the *k* categories for each level in the tree   
The aim of moving to a category-based solution is to embed knowledge of the taxonomy into classifiers. To do this, I had to figure out how to get positive and negative samples for each category. For every category node, I decided that itself, as well as all its descendants, were *positive* samples for that class. All other nodes that were neither the categories ancestor(s) or itself were set as *negative* samples.  The figure below gives a visual explanation of selecting category training set. I did this for each category node in the tree that had enough data.
<img src="images/hc_5.png" alt="Drawing" style="width: 450px;"/>

One of the limitations of this method is that most nodes have un-balanced classes. Some more severe than other, especially as you go further down in the tree. There are some classes whose ratio of positive signals is as small as 0.02%. The more granular that subclasses get, the harder it is to classify them. The good news is that one can decide to focus on a few of these classes and use synthetic methods to rebalance the dataset.

In [22]:
# Create a dict of positive and negative samples for the categories
category_positive_samples = {}
category_negative_samples= {}

In [23]:
level = 'level_0'
for key in the_level[level].keys():
    find_name = df_catergory_lookup.category_id == key
    name = df_catergory_lookup[find_name].category_name
    the_index = name.index[0]
    

    print ("processing {0}: {1}".format(key, name.ix[the_index]))
    # create POSITIVE training set
    
    key_df = df[level] == key
    null_examples = df['level_1'].isnull()
    positive_examples = df['level_1'].notnull()
    

    # Create training set 
    try:
        category_positive_samples[key] = df[key_df & null_examples]
        category_positive_samples[key] = category_positive_samples[key].append(df[key_df & positive_examples])
        
    except:
        print("This category didn't generate any feature set {0}".format(key))
        continue
        
    # create NEGATIVE training set
    level_0_item = df['level_0'] != key
    
    # Create training set 
    try:
        category_negative_samples[key] = df[level_0_item]
        
    except:
        print("This category didn't generate any label set {0}".format(key))
        continue
        
    
    
print("Finished Processing")




processing 1: Food
processing 2: Supplies
processing 3: Beverages
processing 4: Other
processing 495: Grocery
processing 499: Protein
Finished Processing


In [24]:
level = 'level_2'
for key in the_level[level].keys():
    find_name = df_catergory_lookup.category_id == key
    name = df_catergory_lookup[find_name].category_name
    the_index = name.index[0]

    print ("processing {0}: {1}".format(key, name.ix[the_index]))
    # create POSITIVE training set
    
    key_df = df[level] == key
    null_examples = df['level_3'].isnull()
    positive_examples = df['level_3'].notnull()
    

    # Create training set 
    try:
        category_positive_samples[key] = df[key_df & null_examples]
        category_positive_samples[key] = category_positive_samples[key].append(df[key_df & positive_examples])    
    except:
        print("This category didn't generate any feature set {0}".format(key))
        continue
        
    # create NEGATIVE training set
    find_level_0 = df[level] == key
    name_level_0 = df[find_level_0].level_0
    the_index_level_0 = name_level_0.index[0]
    
    find_level_1 = df[level] == key
    name_level_1 = df[find_level_1].level_1
    the_index_level_1 = name_level_1.index[0]
    
    level_0_item = df['level_0'] != name_level_0.ix[the_index_level_0]
    level_1_item = df['level_1'] != name_level_1.ix[the_index_level_1]
    level_2_item = df['level_2'] != key
    
    # Create training set 
    try:
        category_negative_samples[key] = df[level_0_item & level_1_item & level_2_item]
        
    except:
        print("This category didn't generate any label set {0}".format(key))
        continue    
    
print("Finished Processing")



processing 1281: SF Checkout Bag Fee
processing 130: Cream
processing 375: Sausages
processing 132: Tomatoes
processing 134: Peppers
processing 391: Salt
processing 392: Disposables & Packaging Supplies
processing 267: Glasses
processing 258: Keg Deposit
processing 143: Tortillas
processing 16: Beers
processing 145: Cucumbers
processing 18: Wines
processing 531: Condiments
processing 535: Fruits
processing 152: Flour & Starch
processing 537: Herbs
processing 26: Juices
processing 157: Garlic
processing 1030: Seeds
processing 40: Teas
processing 44: Beans
processing 482: Linens
processing 177: Onions
processing 51: Fuel & Freight/Delivery
processing 1077: Breads
processing 54: Liquor
processing 55: Oils
processing 56: Pork
processing 1730: Leaves
processing 325: Eggs
processing 70: Beef
processing 456: Sodas
processing 329: Poultry
processing 74: Fish
processing 331: Coffee
processing 76: Spices
processing 226: Vinegars
processing 78: Cheese
processing 464: Canned
processing 1272: Squas

In [25]:
level = 'level_3'
for key in the_level[level].keys():
    find_name = df_catergory_lookup.category_id == key
    name = df_catergory_lookup[find_name].category_name
    the_index = name.index[0]

    print ("processing {0}: {1}".format(key, name.ix[the_index]))
    # create POSITIVE training set
    
    key_df = df[level] == key
    null_examples = df['level_4'].isnull()
    positive_examples = df['level_4'].notnull()
    

    # Create training set 
    try:
        category_positive_samples[key] = df[key_df & null_examples]
        category_positive_samples[key] = category_positive_samples[key].append(df[key_df & positive_examples])   
    except:
        print("This category didn't generate any feature set {0}".format(key))
        continue
        
    # create NEGATIVE training set
    find_level_0 = df[level] == key
    name_level_0 = df[find_level_0].level_0
    the_index_level_0 = name_level_0.index[0]
    
    find_level_1 = df[level] == key
    name_level_1 = df[find_level_1].level_1
    the_index_level_1 = name_level_1.index[0]
    
    find_level_2 = df[level] == key
    name_level_2 = df[find_level_2].level_1
    the_index_level_2 = name_level_2.index[0]
    
    level_0_item = df['level_0'] != name_level_0.ix[the_index_level_0]
    level_1_item = df['level_1'] != name_level_1.ix[the_index_level_1]
    level_2_item = df['level_2'] != name_level_2.ix[the_index_level_2]
    level_3_item = df['level_3'] != key
    
    # Create training set 
    try:
        category_negative_samples[key] = df[level_0_item & level_1_item & level_2_item & level_3_item]
        
    except:
        print("This category didn't generate any label set {0}".format(key))
        continue    
    
print("Finished Processing")



processing 387: Oysters
processing 136: Cilantro
processing 1033: Ground Black Pepper
processing 1802: Baby Carrot
processing 1239: Lids
processing 275: Containers
processing 148: Olives
processing 25: Vodka
processing 282: Uniforms
processing 286: Bags
processing 32: Gin
processing 289: Cups
processing 164: Mints
processing 39: Tequila
processing 176: Oranges
processing 1160: Red Wines
processing 50: Iced Tea
processing 52: Whiskey
processing 437: Bacon
processing 1209: Gloves
processing 319: Red Onions
processing 963: Mozzarella
processing 69: Chicken
processing 970: Extra Virgin Olive Oil
processing 1200: Shrimp
processing 1358: Rice
processing 80: Apples
processing 355: Duck
processing 980: Dried Fruits
processing 983: Cheddar
processing 88: Avocados
processing 1113: Kegs
processing 95: Basil
processing 99: Bell Peppers
processing 229: Sauces
processing 998: White Wines
processing 363: Rum
processing 403: Purees
processing 247: Zucchini Squash
processing 1019: Salmon
processing 114

In [26]:
level = 'level_4'
for key in the_level[level].keys():
    find_name = df_catergory_lookup.category_id == key
    name = df_catergory_lookup[find_name].category_name
    the_index = name.index[0]

    print ("processing {0}: {1}".format(key, name.ix[the_index]))
    # create POSITIVE training set
    
    key_df = df[level] == key
    null_examples = df['level_5'].isnull()
    positive_examples = df['level_5'].notnull()
    

    # Create training set 
    try:
        category_positive_samples[key] = df[key_df & null_examples]
        category_positive_samples[key] = category_positive_samples[key].append(df[key_df & positive_examples])
        
    except:
        print("This category didn't generate any feature set {0}".format(key))
        continue
        
    # create NEGATIVE training set
    find_level_0 = df[level] == key
    name_level_0 = df[find_level_0].level_0
    the_index_level_0 = name_level_0.index[0]
    
    find_level_1 = df[level] == key
    name_level_1 = df[find_level_1].level_1
    the_index_level_1 = name_level_1.index[0]
    
    find_level_2 = df[level] == key
    name_level_2 = df[find_level_2].level_1
    the_index_level_2 = name_level_2.index[0]
    
    find_level_3 = df[level] == key
    name_level_3 = df[find_level_3].level_1
    the_index_level_3 = name_level_3.index[0]
    
    level_0_item = df['level_0'] != name_level_0.ix[the_index_level_0]
    level_1_item = df['level_1'] != name_level_1.ix[the_index_level_1]
    level_2_item = df['level_2'] != name_level_2.ix[the_index_level_2]
    level_3_item = df['level_3'] != name_level_3.ix[the_index_level_3]
    level_4_item = df['level_4'] != key
    
    # Create training set 
    try:
        category_negative_samples[key] = df[level_0_item & level_1_item & level_2_item & level_3_item & level_4_item]
        
    except:
        print("This category didn't generate any label set {0}".format(key))
        continue    
    
print("Finished Processing")



processing 35: Bourbon
processing 999: Cabernet Sauvignon
processing 1064: Chardonnay
processing 1002: Sauvignon Blanc
processing 1164: Brut
processing 49: Rye
processing 21: Rose Wine
processing 1015: Chicken Breast
processing 989: Pinot Noir
Finished Processing


In [27]:
len(category_positive_samples.keys())

122

In [28]:
category_positive_samples[35].tail(3)

,mapped_category,item_id,price_stddev,primary_unit,price_mean,category,vendor_id,item_name,level_0,level_1,...,catg_986,catg_989,catg_998,catg_999,catg_1002,catg_495,catg_499,catg_1015,catg_1019,catg_1021
125929,"[Beverages, Alcoholic, Liquor, Whiskey, Bourbo...",610087,None,1,182.4,"[3, 6, 54, 52, 35, 2439]",9396,jim beam rye 90,3,6.0,...,0.258597,NaN,NaN,NaN,NaN,0.259861,0.283328,0.000254,0.150631,0.175491
126081,"[Beverages, Alcoholic, Liquor, Whiskey, Bourbo...",612831,None,21,35.95,"[3, 6, 54, 52, 35, 2426]",9530,basil haydens bourbon 80,3,6.0,...,0.324394,NaN,NaN,NaN,NaN,0.310331,0.283328,0.000254,0.181986,0.348535
126665,"[Beverages, Alcoholic, Liquor, Whiskey, Bourbo...",528004,None,21,45.8,"[3, 6, 54, 52, 35, 2481]",10335,breaker bbn port finish 90,3,6.0,...,0.355630,NaN,NaN,NaN,NaN,0.281260,0.247167,0.011750,0.214430,0.353754


In [29]:
category_negative_samples[35].head(3)

,mapped_category,item_id,price_stddev,primary_unit,price_mean,category,vendor_id,item_name,level_0,level_1,...,catg_986,catg_989,catg_998,catg_999,catg_1002,catg_495,catg_499,catg_1015,catg_1019,catg_1021
0,"[Supplies, Restaurant Supplies, Disposables & ...",167,7.55423e-07,1,28.7,"[2, 12, 392, 430]",5,fork 10in serving black 144/cs,2,12.0,...,0.264482,0.288393,0.363877,0.251602,0.230086,0.264524,0.180126,0.00914,0.166362,0.306131
1,"[Food, Meats, Pork]",176,6.06195e-08,3,2.19,"[1, 7, 56]",7,bacon ends 15# (kruse),1,7.0,...,0.202777,0.197688,0.264600,0.214336,0.183216,0.285991,0.331882,NaN,NaN,NaN
2,"[Food, Meats, Beef]",178,4.11051e-07,3,8.45,"[1, 7, 70]",7,flat iron steak ready choice,1,7.0,...,0.355630,0.286951,0.285717,0.280367,0.197030,0.288223,0.306360,NaN,NaN,NaN


## Modeling

With the training sets done, I vectorized them using a mean embedding vectorizer. For each category, I created an AdaBoost binary classifier. I chose to use the AdaBoost after I experimented with several classifiers like LinearSVM and Decision Trees. I first selected Decision Tree since it outperformed the LinearSVM for this problem. However, I quickly realized my aim was to get predicted probabilities as output for the class instead of simple binary output which the Decision Tree gave. For that reason, I switched to AdaBoost, which is a *boosted* decision tree. 

I train each classifier using a k=5 kfold cross-validation scheme. I fitted the resulting classifier and retrieved the predicted probability for each data point in the dataset, which resulted in *k* vectors for *k* categories.

#### Process item label names using Word2Vec

In [30]:
from gensim.models.word2vec import Word2Vec

# Only use item labels as input into word2vec embeddings
# train word2vec on all the item_labels 

w2v_model = Word2Vec(df['item_labels'], size=750, window=5, min_count=5, workers=4)
w2v = dict(zip(w2v_model.index2word, w2v_model.syn0))


In [31]:
w2v_model.init_sims(replace=True) #to trim unneeded model memory = use (much) less RAM.

#### Train *k* classifiers

In [32]:
from time import time
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import MinMaxScaler







seed = 342 # For reproducability
Ada_w2v =  Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)), ("Ada", AdaBoostClassifier(n_estimators=10))])


In [33]:
try:
    with open('data/df_category_scores.json') as f:    
        scores = json.load(f)
except:
    scores = {}

In [ ]:
try:
    pred_proba = pd.read_pickle('data/df_category_pred_proba.pkl')
except:
    pred_proba = {}

In [ ]:
t0 = time()
target = open('data/training_output.txt', 'w')
print ("processing {0}: {1}".format(key, name.ix[the_index]))
        

for key in category_positive_samples:
    if str(key) in scores:
        continue
    if key in scores:
        continue
    else:
        find_name = df_catergory_lookup.category_id == key
        name = df_catergory_lookup[find_name].category_name
        the_index = name.index[0]

        target.write("processing {0}: {1}\n".format(key, name.ix[the_index]))
        category_positive_samples[key].loc[:,('is_category')] = 1
        category_negative_samples[key].loc[:,('is_category')] = 0
        
        # Split into training and testing set
        data = category_positive_samples[key].append(category_negative_samples[key], ignore_index=True)
        X = data['item_labels']
        y = data['is_category']
        target.write('{:5.3f}% positive samples\n'.format(len(category_positive_samples[key])/len(X)))
        target.write("Number of datapoints in set {}\n".format(len(X)))

        target.write('Training the model....\n')

        scores[key] = cross_val_score(Ada_w2v,  X, y, cv=5, scoring='f1').mean()
        Ada_w2v.fit(X, y)
        pred_proba[key] = Ada_w2v.predict_proba(X)

        print('Finished training category{0}: {1}\n\n'.format(key, name.ix[the_index]))
        target.write('Finished training category{0}: {1}\n\n'.format(key, name.ix[the_index]))
        
target.close()    
print("done in %0.3fs" % (time() - t0))
json.dump(scores, open("data/df_category_scores.json",'w'))



processing 989: Pinot Noir


It took 14 hours to run

## Get *k* vectors

In [ ]:
for key in category_positive_samples:
    df_ = category_positive_samples[key].append(category_negative_samples[key])
    scores_ = pd.DataFrame(pred_proba[key], columns = ['Neg','catg_'+str(key)])
    scores_.drop(['Neg'], axis = 1, inplace = True)
    scores_ = scores_.set_index(df_.index)
    df.ix[df_.index, 'catg_'+str(key)] = scores_

### Write vectors to file

In [ ]:
df.to_pickle('data/df_data_vectors.dat')
pickle.dump(pred_proba, open("data/df_category_pred_proba.pkl",'wb'))


### Final Model Architecture
<figure>
  <img src="images/hc_4.png" style="width: 450px;">
</figure>
After training the *k=141* classifiers, I extracted the *k* vectors; I carefully combined them with the engineered features, and the other raw metadata taking care to ensure that I assigned the right probabilities to the right data points in my training set. I feed these features and labels into a multi-label, multi-class AdaBoost classifier. 

I feed these features and labels into a multi-label, multi-class AdaBoost classifier. I took 75% of the data for training and 25% for testing. Once again I do a 5 fold cross-validation scheme, fit the final classifier, and retrieve the predicted class probabilities.


#### Get Data for Final Classifier

In [ ]:
the_indices = {}

In [ ]:
for key in category_positive_samples:
    key_df = df['label'] == key
    the_indices[key] = df[key_df].index

In [ ]:
X = pd.DataFrame()

In [ ]:
for key in the_indices:
    X = X.append(pd.DataFrame(df.loc[the_indices[key]], columns = df.keys()))

In [ ]:
X.drop([u'mapped_category', u'category',
u'item_name',         u'level_0',
u'level_1',         u'level_2',         u'level_3',
u'level_4',         u'level_5',         u'level_6',
u'mapped_level_0',  u'mapped_level_1',  u'mapped_level_2',
u'mapped_level_3',  u'mapped_level_4',  u'mapped_level_5',
u'mapped_level_6',   u'item_labels'], axis = 1, inplace=True)

In [ ]:

X.head()

#### Drop all rows whose label is NaN

In [ ]:
X = X[X['label'].notnull()]
X = X.fillna(0)

In [ ]:
y = X['label']
X.drop(['label'], axis = 1, inplace = True)

In [ ]:
print("Number of datapoints in set {}".format(len(X)))
# First, decide how many training vs test samples you want
num_train = ( len(y) // 4) *3

# Split the data into training and testing sets

try:
    X_train, y_train, X_test, y_test = shuffle_split_data(X, y)
    print ("Successfully shuffled and split the data!")
except:
    print ("Something went wrong with shuffling and splitting the data.")


## Train final classifier

In [ ]:
models = {
          'XGBoost': XGBClassifier(),
          'DecisionTree': DecisionTreeClassifier(random_state=seed),
          'SVC': LinearSVC(random_state=seed),
          'RandomForest': RandomForestClassifier(random_state=seed),
          'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=seed)
         }



scaler = MinMaxScaler()
X_transform = scaler.fit_transform(X_train)

print('CLASSIFICATION RESULTS OF BASELINE CLASSIFIERS\n')
print('{:20}{:^15}{:^10}'.format('CLASSIFIER', 'MEAN SCORE %', 'STD DEV %'))


for clf_name, clf in models.iteritems():
    results = cross_val_score(clf, X_transform, y_train, cv=5)
    print('{:20}{:^15.2f}{:^10.2f}'.format(clf_name, results.mean()*100, results.std()*100) ) 
    

In [ ]:

clf = XGBClassifier()
clf.fit(X_train_trans, y_train)
X_test_trans = scaler.transform(X_test)
final_preds = clf.predict(X_test_trans)
precision, recall, fbeta_score, support = score(y_test, final_preds)


print ("Precision: {:15.3f}\nRecall: {:^10.3f}\nF Score{:^10.3f}\nSupport:{:^10.3f}".format(precision.mean()*100, 
                                                 recall.mean()*100, fbeta_score.mean()*100, len(support)))


In [ ]:
y_test.unique()

In [ ]:
score(y_test, final_preds)

In [ ]:
C = confusion_matrix(y_test, final_preds)

In [ ]:
print (clf.classes_)
C


In [ ]:
a = df_catergory_lookup['category_id'] == 263
df_catergory_lookup[a]

# Result
With the understanding that chance is 1/141 * 100 = 0.7%, my approach achieves a Precision of 17%, Recall of 19% and FScore of 15% for the last level of the tree. 


In [ ]:
for key in category_positive_samples:
        find_name = df_catergory_lookup.category_id == key
        name = df_catergory_lookup[find_name].category_name
        the_index = name.index[0]

        

In [ ]:
outX = pd.DataFrame(index=y_test.index)
outX = outX.join(y_test)
outX['predicted'] = final_preds
outX['predicted_name'] = ''
outX['item_name'] = pd.DataFrame(df.loc[outX.index])['item_name']
outX['mapped_level_0'] = pd.DataFrame(df.loc[outX.index])['mapped_level_0']
outX['level_0'] = pd.DataFrame(df.loc[outX.index])['level_0']
outX['mapped_level_1'] = pd.DataFrame(df.loc[outX.index])['mapped_level_1']
outX['level_1'] = pd.DataFrame(df.loc[outX.index])['level_1']
outX['mapped_level_2'] = pd.DataFrame(df.loc[outX.index])['mapped_level_2']
outX['level_2'] = pd.DataFrame(df.loc[outX.index])['level_2']
outX['mapped_level_3'] = pd.DataFrame(df.loc[outX.index])['mapped_level_3']
outX['level_3'] = pd.DataFrame(df.loc[outX.index])['level_3']
outX['mapped_level_4'] = pd.DataFrame(df.loc[outX.index])['mapped_level_4']
outX['level_4'] = pd.DataFrame(df.loc[outX.index])['level_4']
outX['mapped_level_5'] = pd.DataFrame(df.loc[outX.index])['mapped_level_5']
outX['level_5'] = pd.DataFrame(df.loc[outX.index])['level_5']

In [ ]:
j = 0
for i in final_preds:
    find_name = df_catergory_lookup.category_id == i
    outX.ix[outX.index[j], 'predicted_name'] = df_catergory_lookup[find_name]['category_name'].values[0]
    j += 1

In [ ]:
#score(y_test, final_preds)
outX.head()

In [ ]:
# Given a category find ancestors


### Classification error by tree depth

In [ ]:

df_lookup = df[['level_0', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5',
                        'mapped_level_0', 'mapped_level_1', 'mapped_level_2', 'mapped_level_3',
                        'mapped_level_4', 'mapped_level_5']]


In [ ]:
key = 18
a = (df_lookup['level_0'] == key)
b = (df_lookup['level_1'] == key)
c = (df_lookup['level_2'] == key)
d = (df_lookup['level_3'] == key)
e = (df_lookup['level_3'] == key)
df_lookup[ a | b | c | d | e ][:1]

In [ ]:
category_positive_samples[235].head()

#### Retrain classifier for category k = 235, chocolate
The value of this method is that I can take a look at the results, determine which individual text-based classifiers have poor results, and optimize and tune that specific classifier. In the case of the chocolate classifier, I see that the minority class is minuscule. To deal with this great imbalance in the data, I synthetically resample it and retrain the classifier.

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE 

key = 235

data = category_positive_samples[key].append(category_negative_samples[key], ignore_index=True)
X_235 = data['item_labels']
y_235 = data['is_category']
        
vectorizer = MeanEmbeddingVectorizer(w2v)
vectorizer.fit(X_235, y_235)  

sm = SMOTE(random_state=seed)

print('Original dataset shape {}'.format(Counter(y_235)))
X_res, y_res = sm.fit_sample(vectorizer.transform(X_235) , y_235)
print('Resampled dataset shape {}'.format(Counter(y_res)))

In [ ]:
print('Training the model....')
clf = AdaBoostClassifier(n_estimators=10)
scores_235 = cross_val_score(clf, X_res, y_res, cv=5, scoring='f1').mean()
clf.fit(X_res, y_res)
pred_proba_235 = clf.predict_proba(X_res)
print('Finished training category\n\n')

In [ ]:
X_ = vectorizer.transform(X_235)
clf.fit(X_, y_235)
pred_proba_235 = clf.predict_proba(X_)

In [ ]:
scores[key] = scores_235


In [ ]:
key = 235
df_ = category_positive_samples[key].append(category_negative_samples[key])
scores_ = pd.DataFrame(pred_proba_235, columns = ['Neg','catg_'+str(key)])
scores_.drop(['Neg'], axis = 1, inplace = True)
scores_ = scores_.set_index(df_.index)
df.ix[df_.index, 'catg_'+str(key)] = scores_

## Calculate chance and scores for each level in the tree